# En este ejemplo vamos a construir un React Agent desde 0.
Vamos a por fin quitarnos toda la abstraccion de los Agentes de IA y vamos a ver las entranas. 

Como conseguimos que un modelo de IA, sea capaz de razonar y utilizar las herramientas que tiene a su disposicion para cumplir su objetivo a traves de varios pasos?

Para ello vamos a utilizar el ReAct pattern.

## Importamos las dependencias y el cliente de Groq

In [1]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq

from tool import tool
from utils.extraction import extract_tag_content


# cargamos las variables de entorno, ahi debera estar nuestra API de Groq
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

Al igual que hicimos con el Patrón de Herramientas (Tool Pattern), también necesitamos un System Prompt para la técnica ReAct. Este System Prompt es muy similar, pero la diferencia radica en que describe el bucle ReAct, de modo que el LLM sea consciente de las tres operaciones que puede utilizar:

Pensamiento (Thought): El LLM reflexionará sobre qué acción tomar.

Acción (Action): El LLM utilizará una herramienta para "actuar sobre el entorno".

Observación (Observation): El LLM observará el resultado de la herramienta y reflexionará sobre el siguiente paso a seguir.

Vamos a utilizar XML prompting, una tecnica que hace mucho mas precisas las respuestas de los modelos.




In [2]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

# Ejemplo paso a paso
Vamos a definir 3 herramientas simples

In [3]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}


Recuerda que el operador @tool nos permite convertir una función de Python en una Herramienta de forma automática. Podemos comprobarlo muy fácilmente con algunas de las funciones anteriores.

In [4]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


In [5]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [6]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"prop

In [7]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [8]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

### ReAct Loop Paso 1

1. hacemos la pregunta y la incorporamos al historial de mensajes junto con el system prompt
2. Enviamos el chat de mensajes al LLM 
3. Guardamos la respuesta en el historial de mensajes


In [9]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [10]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result. I will first call the sum_two_elements function with a = 1234 and b = 5678.</thought>
<tool_call>{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>


In [11]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Paso 2

1. Extraemos la llamada a la funcion a partir del output del modelo
2. Ejecutamos la funcion
3. Recogemos el resultado y lo incorporamos al historial de mensajes
   


In [12]:
tool_call = extract_tag_content(output, tag="tool_call")

In [13]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}'], found=True)

In [14]:
tool_call = json.loads(tool_call.content[0])

In [15]:
tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [16]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [17]:
assert tool_result == 1234   + 5678

In [18]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Paso 3

1. Volvemos a llamar al modelo con el historial de mensajes (contiene la respuesta de la funcion)
2. Guardamos la respuesta en el historial de mensajes

In [19]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5. I will call the multiply_two_elements function with a = 6912 and b = 5.</thought>
<tool_call>{"name": "multiply_two_elements","arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [20]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Paso 4
1. Lo mismo que antes, volvemos a extraer la llamada a la funcion
2. Ejecutamos la funcion 
3. Guardamos el resultado de la funcion en el historial de mensajes

In [21]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [22]:
tool_result

34560

In [23]:

assert tool_result == (1234 + 5678) * 5

In [24]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Paso 5

volvemos a llamar al modelo con el historial de mensajes, y anadimos la respuesta al historial

In [25]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the result of multiplying 6912 by 5, which is 34560. Now, I need to compute the logarithm of this result. I will call the compute_log function with x = 34560.</thought>
<tool_call>{"name": "compute_log","arguments": {"x": 34560}, "id": 2}</tool_call>


In [26]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Paso 6

1. volvemos  extraer la llamada a la funcion 
2. Ejecutamos la funcion
3. Guardamos el resultado en el historial de mensajes

In [27]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [28]:
tool_result

10.450452222917992

In [29]:
assert tool_result == math.log((1234 + 5678) * 5)

In [30]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

ReAct Loop Paso 7

En este caso para resolver el problema, debiamos ejecutar 3 funciones, por lo que hemos iterado sobre el modelo 3 veces, posteriormente automatizaremos este proceso de forma que sea el modelo el que decida el numero de veces que debe repetir el loop, hasta encontrar la respuesta.


In [31]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the logarithm of 34560, which is approximately 10.45. I have completed all the calculations requested by the user.</thought>
<response>The sum of 1234 and 5678 is 6912. Multiplying 6912 by 5 results in 34560. The logarithm of 34560 is approximately 10.45.</response>


Hacemos lo mismo pero de forma automatica

In [33]:

from react_agent import ReactAgent

In [39]:

agent = ReactAgent(
    model="llama-3.3-70b-versatile",
    tools=[sum_two_elements, multiply_two_elements, compute_log]
    )

In [40]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")


Thought: I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result.

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}

Thought: I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}

Thought: I have the result of multiplying 6912 by 5, which is 34560. Now, I need to compute the logarithm of this result.

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 2}

Tool result: 
10.450452222917992

Observations: {2: 10.450452222917992}


'The sum of 1234 and 5678 is 6912. Multiplying 6912 by 5 results in 34560. The logarithm of 34560 is approximately 10.45.'